<a href="https://colab.research.google.com/github/gsaveri/SinGAN/blob/master/SinGan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#SinGAN Colab Notebook

This repo was originally forked from https://github.com/tamarott/SinGAN, I just added the colab notebook.

[Project](https://tamarott.github.io/SinGAN.htm) | [Arxiv](https://arxiv.org/pdf/1905.01164.pdf) | [CVF](http://openaccess.thecvf.com/content_ICCV_2019/papers/Shaham_SinGAN_Learning_a_Generative_Model_From_a_Single_Natural_Image_ICCV_2019_paper.pdf) | [Supplementary materials](https://openaccess.thecvf.com/content_ICCV_2019/supplemental/Shaham_SinGAN_Learning_a_ICCV_2019_supplemental.pdf) | [Talk (ICCV`19)](https://youtu.be/mdAcPe74tZI?t=3191) 


### Official pytorch implementation of the paper: "SinGAN: Learning a Generative Model from a Single Natural Image"
#### ICCV 2019 Best paper award (Marr prize)

The code will clone the repository into your local colab space, and train sinGan with an input image.

### Citation
If you use this code for your research, please cite the paper:

```
@inproceedings{rottshaham2019singan,
  title={SinGAN: Learning a Generative Model from a Single Natural Image},
  author={Rott Shaham, Tamar and Dekel, Tali and Michaeli, Tomer},
  booktitle={Computer Vision (ICCV), IEEE International Conference on},
  year={2019}
}
```


##Code

In [ ]:
import os
from google.colab import files

In [ ]:
from google.colab import drive
from pathlib import Path

content_path = Path('/').absolute() / 'content'
drive_path = content_path / 'drive'
drive.mount(str(drive_path))

### Clone the repository

In [ ]:
singan_repo_url  = 'https://github.com/gsaveri/SinGAN.git'
project_path        = drive_path / 'MyDrive' / 'SinGan'
singan_repo_path = project_path / 'SinGan'

# Create project folder if inexistant
if not project_path.is_dir():
    %mkdir "{project_path}"
%cd "{project_path}"

if singan_repo_path.is_dir():
    !git -C "{singan_repo_path}" fetch origin
    !git -C "{singan_repo_path}" checkout origin/main -- *.py
else:
    print("Install SinGan")
    !git clone {singan_repo_url}

In [ ]:
cd /content/drive/MyDrive/SinGan

### Install dependencies

This code was tested with python 3.6  

In [ ]:
!pip install -r requirements.txt

### Train on uploaded image

In [ ]:
# Load an input image
input = files.upload()
input = list(input.keys())[0]

!cp "{input}" /content/SinGAN/Input/Images/"{input}"
!rm "{input}"

In [ ]:
!python main_train.py --input_name "{input}"

###  Random samples
To generate random samples from any starting generation scale, please first train SinGAN model on the desired image (as described above), then run

In [ ]:
gen_start_scale = 0

In [ ]:
folder = input[0:len(input)-4]
print(folder)

In [ ]:
! rm -r /content/SinGAN/Output/RandomSamples/seed0936/gen_start_scale=0

In [ ]:
!python random_samples.py --input_name "{input}" --mode random_samples --gen_start_scale "{gen_start_scale}"

In [ ]:
!python random_samples.py --input_name "{input}" --mode random_samples_arbitrary_sizes --scale_h 1 --scale_v 1

In [ ]:
#Copy to google drive

from os import path
singan = '/content/drive/MyDrive/Side Hustles/saivnnt/singan'
random_samples_folder = path.join(singan,folder)
#source = path.join('/content/SinGAN/Output/RandomSamples',folder,'gen_start_scale=' + str(gen_start_scale))
source = '/content/SinGAN/Output/RandomSamples_ArbitrerySizes/seed0936/scale_v=4.000000_scale_h=4.000000'
print(random_samples_folder)

!mkdir "{random_samples_folder}"
!cp "{source}"/*.* "{random_samples_folder}"

### Harmonization

To harmonize a pasted object into an image (See example in Fig. 13 in [our paper](https://arxiv.org/pdf/1905.01164.pdf)), please first train SinGAN model on the desired background image (as described above), then save the naively pasted reference image and it's binary mask under "Input/Harmonization" (see saved images for an example). Run the command

In [ ]:
# Load paint reference image
harmonization_ref = files.upload()
harmonization_ref = list(harmonization_ref.keys())[0]

harmonization_mask = files.upload()
harmonization_mask = list(harmonization_mask.keys())[0]

!cp "{harmonization_ref}" /content/SinGAN/Input/Harmonization/"{harmonization_ref}"
!cp "{harmonization_mask}" /content/SinGAN/Input/Harmonization/"{harmonization_mask}"
!rm "{harmonization_ref}"
!rm "{harmonization_mask}"

In [ ]:
!python harmonization.py --input_name "{input}" --ref_name  "{harmonization_ref}" --harmonization_start_scale 7

Please note that different injection scale will produce different harmonization effects. The coarsest injection scale equals 1. 

###  Editing

To edit an image, (See example in Fig. 12 in [our paper](https://arxiv.org/pdf/1905.01164.pdf)), please first train SinGAN model on the desired non-edited image (as described above), then save the naive edit as a reference image under "Input/Editing" with a corresponding binary map (see saved images for an example). Run the command

In [ ]:
# Load an input image
input = files.upload()
input = list(input.keys())[0]

!cp "{input}" /content/SinGAN/Input/Editing/"{input}"
!rm "{input}"

# Load an input image
ref = files.upload()
ref = list(ref.keys())[0]

!cp "{ref}" /content/SinGAN/Input/Editing/"{ref}"
!rm "{ref}"

In [ ]:
!python editing.py --input_name 'seed0365.png' --ref_name seed0365_edit.png --editing_start_scale 5

both the masked and unmasked output will be saved.
Here as well, different injection scale will produce different editing effects. The coarsest injection scale equals 1. 


###Super resolution

To super resolve an image, please run:

In [ ]:
!python SR.py --input_name "{input}"

This will automatically train a SinGAN model correspond to 4x upsampling factor (if not exist already).
For different SR factors, please specify it using the parameter `--sr_factor` when calling the function.
SinGAN's results on the BSD100 dataset can be download from the 'Downloads' folder.

### Paint to image

To transfer a paint into a realistic image (See example in Fig. 11 in [our paper](https://arxiv.org/pdf/1905.01164.pdf)), please first train SinGAN model on the desired image (as described above), then save your paint under "Input/Paint", and run the command

In [ ]:
# Load paint reference image
paint_ref = files.upload()
paint_ref = list(input.keys())[0]

!cp "{paint_ref}" /content/SinGAN/Input/Paint/"{paint_ref}"
!rm "{paint_ref}"

In [ ]:
!python paint2image.py --input_name "{input}" --ref_name "{paint_ref}" --paint_start_scale 2

Here as well, different injection scale will produce different editing effects. The coarsest injection scale equals 1. 

Advanced option: Specify quantization_flag to be True, to re-train *only* the injection level of the model, to get a on a color-quantized version of upsampled generated images from the previous scale. For some images, this might lead to more realistic results.

In [ ]:
# clean trainedModel
# !rm -r /content/SinGAN/Output/RandomSamples/"{folder}"
# !rm -r /content/SinGAN/TrainedModels/"{folder}"

In [ ]:
## Additional Data and Functions

### Single Image Fréchet Inception Distance (SIFID score)
To calculate the SIFID between real images and their corresponding fake samples, please run:



In [ ]:
# !python SIFID/sifid_score.py --path2real <real images path> --path2fake <fake images path>   

Make sure that each of the fake images file name is identical to its corresponding real image file name. Images should be saved in `.jpg` format.

### Super Resolution Results
SinGAN's SR results on the BSD100 dataset can be download from the 'Downloads' folder.

### User Study
The data used for the user study can be found in the Downloads folder. 

real folder: 50 real images, randomly picked from the [places database](http://places.csail.mit.edu/)

fake_high_variance folder: random samples starting from n=N for each of the real images 

fake_mid_variance folder: random samples starting from n=N-1 for each of the real images 

For additional details please see section 3.1 in our [paper](https://arxiv.org/pdf/1905.01164.pdf)